In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
data = pd.read_csv("vectorized_fw2.csv", index_col=0)
test = pd.read_csv("test_ds_fw2.csv", index_col=0)

In [ ]:
wc_t = data["total wc"][0]

In [ ]:
(data.iloc[:,3:]/wc_t).plot(legend = False)
plt.xlabel("text label (grouped by judges)")
plt.ylabel("word frequency")
plt.savefig("freq")

In [ ]:
train_x = np.array(data.iloc[:,3:])
train_x = np.nan_to_num(train_x)
train_x = train_x/wc_t
train_y = np.array(data.iloc[:,0])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=2, criterion = "entropy")

In [ ]:
clf.fit(train_x, train_y)

In [ ]:
wc = np.array(test["total wc"])
temp = np.array(test.iloc[:,3:])
temp = np.nan_to_num(temp)
test_x = []
for i in range(wc.shape[0]):
    vec = temp[i]/wc[i]
    test_x.append(vec)
test_x = np.array(test_x)

In [ ]:
feature_importances = pd.Series(clf.feature_importances_, index=data.columns[3:])

In [ ]:
feature_importances.sort_values(ascending=False, inplace=True)

In [ ]:
feature_importances.to_csv("feature_importances.csv")

In [ ]:
def judge_list(n):
    if n in range(573,609+1):
        return ["楊仁壽","城仲模","翁岳生","林永謀","王和雄","余雪明","廖義男","曾有田","彭鳳至","賴英照","謝在全","徐璧湖","林子儀","許宗力","許玉秀"]
    elif n == 610:
        return ["城仲模","翁岳生","林永謀","王和雄","余雪明","廖義男","曾有田",'彭鳳至',"賴英照","謝在全","徐璧湖","林子儀","許宗力","許玉秀"]

In [ ]:
y = []
y_prob = []
for row in test.iterrows():
    jl = judge_list(row[1]["file_n"])
    clf = RandomForestClassifier(n_estimators=100, n_jobs=2, criterion = "entropy")
    idx_to_train = data.NAME.isin(jl)
    clf.fit(train_x[idx_to_train], train_y[idx_to_train])
    y.append(clf.predict(test_x[row[0]]))
    y_prob.append(clf.predict_proba(test_x[row[0]]))
y = pd.Series(y, index=test.file_n)
y_prob = pd.DataFrame(y_prob, index=test.file_n.as_matrix(), columns=clf.classes_)

In [ ]:
y.to_csv("predict.csv", encoding="utf8")
y_prob.to_csv("predict_proba.csv", encoding="utf8")